In [1]:
from collections import defaultdict
from enum import Enum
from itertools import permutations
import numpy as np
import pickle
import re
import os
from typing import Dict, List, NamedTuple, Optional, Tuple, DefaultDict
from tqdm import tqdm, trange

In [2]:
class Author(Enum):
    PRUS = 0
    SIENKIEWICZ = 1
    ORZESZKOWA = 2

In [3]:
def preprocess(text: str) -> str:
    text = text.lower()
    text = text.strip()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\ +', ' ', text)
    return text

def load_corpus_list(path: str) -> List[str]:
    text = open(path, 'r').read()
    preprocessed = preprocess(text)
    words = preprocessed.split()
    return words

def occurence_dict_from_corpus(words: List[str]) -> Dict[str, float]:
    unique_words, word_counts = np.unique(words, return_counts=True)
    word_counts_perc = word_counts / np.sum(word_counts)
    return defaultdict(float, zip(unique_words, word_counts_perc))

def create_occurence_dicts() -> Dict[Author, Dict[str, float]]:
    occurence_dicts = dict()
    occurence_dicts[Author.PRUS] = occurence_dict_from_corpus(
        words=load_corpus_list(path='data.nogit/korpus_prusa.txt')
    )
    occurence_dicts[Author.SIENKIEWICZ] = occurence_dict_from_corpus(
        words=load_corpus_list(path='data.nogit/korpus_sienkiewicza.txt')
    )
    occurence_dicts[Author.ORZESZKOWA] = occurence_dict_from_corpus(
        words=load_corpus_list(path='data.nogit/korpus_orzeszkowej.txt')
    )
    return occurence_dicts

In [4]:
def create_word_to_tag_mapping(path: str) -> Dict[str, str]:
    file = open(path, 'r').read()
    return dict(np.array(file.replace('\n', ' ').split()).reshape(-1, 2))

In [5]:
def load_bigrams(
    path: str,
    word_to_tag_mapping: Dict[str, str],
) -> Tuple[
        DefaultDict[str, List[str]],
        DefaultDict[Tuple[str, str], float],
        DefaultDict[str, List[str]],
        DefaultDict[Tuple[str, str], float],
    ]:
    MIN_N = 2

    word_mapping = defaultdict(set)
    word_mapping_count = defaultdict(float)
    tag_mapping = defaultdict(set)
    tag_mapping_count = defaultdict(float)
    
    pbar = tqdm(desc='Loading 2grams', position=0, leave=True, total=59134224)
    with open(path, 'r') as f:
        row = True
        while row:
            row = f.readline().strip()

            pbar.update(1)
            row = row.split()

            if len(row) == 0:
                continue

            cnt, word, following = int(row[0]), row[1], row[2]

            if cnt < MIN_N:
                continue

            word_mapping[word].add(following)
            word_mapping_count[(word, following)] += cnt

            if word not in word_to_tag_mapping or following not in word_to_tag_mapping:
                continue

            tag_word, tag_following = word_to_tag_mapping[word], word_to_tag_mapping[following]
            tag_mapping[tag_word].add(tag_following)
            tag_mapping_count[(tag_word, tag_following)] += cnt
           
        pbar.close()
    
    for word in tqdm(list(word_mapping.keys()), desc='Normalizing mappings', position=0, leave=True):
        word_sum = sum([word_mapping_count[(word, following)] for following in word_mapping[word]])
        for following in word_mapping[word]:
            word_mapping_count[(word, following)] /= word_sum
        
        if word not in word_to_tag_mapping:
            continue
        
        tag_word = word_to_tag_mapping[word]
        tag_sum = sum([tag_mapping_count[(tag_word, tag_following)] for tag_following in tag_mapping[tag_word]])
        if tag_sum < 2:
            continue
        for tag_following in tag_mapping[tag_word]:
            tag_mapping_count[(tag_word, tag_following)] /= tag_sum        
        
    return word_mapping, word_mapping_count, tag_mapping, tag_mapping_count

### Zadanie 1. (3+1+Xp) 
Napisz program, który ustala autora zdania. Autorów jest trójka: Prus, Orzeszkowa, Sienkiewicz (POS), na SKOSie znajdziesz odpowiednie dane uczące. Powinieneś je podzielić na część uczącą i walidacyjną. Część X punktacji zależeć będzie od tego, jak Twój program wypadnie na tle innych programów dla danych testowych (które pojawią się później na SKOSie). Dwie podstawowe metody są następujące:

a) Naive Bayes (będzie na wykładzie 4, powinieneś użyć przynajmniej jednej cechy, która nie jest słowem)

b) Utworzenie trzech modeli językowych i wybór tego, który daje najepszy wynik na klasyfikowanym zdaniu.

Za sprawdzenie dwóch podejść jest punkt premiowy.

In [8]:
def find_author(text: str, occurence_dicts: Dict[Author, Dict[str, float]]) -> Author:
    scores: List[float] = []
    for author, occurence_dict in occurence_dicts.items():
        words = preprocess(text=text).split()
        score = np.sum(np.log([occurence_dict[word] + 1e-5 for word in words]))
        scores.append(score)
        
    author = list(occurence_dicts.keys())[np.argmax(scores)]
    return author

In [9]:
def test_classifier(folder_path: str) -> np.ndarray:
    '''
    Returns confussion matrix
    '''
    occurence_dicts = create_occurence_dicts()
    
    authors_number = len(list(occurence_dicts.keys()))
    confussion_matrix = np.zeros((authors_number, authors_number))
    
    for relative_path in os.listdir(folder_path):
        path = os.path.join(folder_path, relative_path)
        
        if not os.path.isfile(path):
            continue
        
        true_author: Optional[Author] = None
        
        if 'orzeszkowej' in relative_path:
            true_author = Author.ORZESZKOWA
        if 'prusa' in relative_path:
            true_author = Author.PRUS
        if 'sienkiewicza' in relative_path:
            true_author = Author.SIENKIEWICZ
        
        if true_author is None:
            print(f'Could not find a true author of the text! Filename: {relative_path}')
            continue
        
        text = open(path, 'r').read()
        classified_author = find_author(text=text, occurence_dicts=occurence_dicts)
        
        confussion_matrix[true_author.value, classified_author.value] += 1
        
    return confussion_matrix

In [10]:
confussion_matrix = test_classifier(folder_path='testy1.nogit')
authors = list(map(lambda author: author.name, list(Author)))
authors, confussion_matrix, f'Accuracy: {np.diag(confussion_matrix).sum() / confussion_matrix.sum()}'

(['PRUS', 'SIENKIEWICZ', 'ORZESZKOWA'],
 array([[21.,  0.,  0.],
        [15., 12.,  0.],
        [ 0.,  0., 12.]]),
 'Accuracy: 0.75')

### Zadanie 2. (3+Xp) 
W zadaniu tym powinieneś uporządkować (spermutować) ciąg słów, żeby utworzył zdanie. W stosunku do poprzedniej listy mamy następujące różnice:

a) powinieneś wykorzystać tagi słów (na przykład z pliku supertags.txt, link na SKOSie)

b) powinieneś je połączyć ze zwykłymi statystykami bigramowymi (lub, opcjonalnie, z sufiksami)

c) powinieneś wyodrębnić z danych uczących część walidacyjną i dobrać parametry łączenia modeli bazujących na słowach i bazujących na tagach.

Sposób oceny będzie taki sam, jak w przypadku zadania z P1. Wybór zdań do oceny pojawi się przed zajęciami (będą to zdania zawierające od 4 do 8 tokenów, wybrane losowo z części testowej korpusu PolEval)

In [6]:
word_to_tag_mapping = create_word_to_tag_mapping(path='data.nogit/supertags.txt')

In [7]:
if os.path.exists('bigrams_saved/saved.pkl'):
    d = pickle.load(open('bigrams_saved/saved.pkl', 'rb'))
    word_bigrams, word_bigrams_count, tag_bigrams, tag_bigrams_count = \
        d['from_words']['bigrams'], d['from_words']['bigrams_count'], d['from_tags']['bigrams'], d['from_tags']['bigrams_count']

else:
    word_bigrams, word_bigrams_count, tag_bigrams, tag_bigrams_count = \
        load_bigrams(path='data.nogit/poleval_2grams.txt', word_to_tag_mapping=word_to_tag_mapping)

    import pickle
    to_save = {
        'from_words': {'bigrams': word_bigrams, 'bigrams_count': word_bigrams_count},
        'from_tags' : {'bigrams': tag_bigrams,  'bigrams_count': tag_bigrams_count},
        }
    pickle.dump(
        to_save,
        open('bigrams_saved/saved.pkl', 'wb+'),
        pickle.HIGHEST_PROTOCOL,
    )

In [59]:
def compute_sentence_naturalness(sentence: List[str], alpha: float) -> Tuple[float, float, float]:
    word_naturalness = tag_naturalness = overall_naturalness = 0
    for i in range(1, len(sentence)):
        word, following = sentence[i-1:i+1]
        word_naturalness += word_bigrams_count[(word, following)]
        
        if word not in word_to_tag_mapping or following not in word_to_tag_mapping:
            continue
        
        tag_word, tag_following = word_to_tag_mapping[word], word_to_tag_mapping[following]
        tag_naturalness += tag_bigrams_count[(tag_word, tag_following)]
        
        overall_naturalness = word_naturalness * alpha + tag_naturalness * (1 - alpha)
        
    return word_naturalness, tag_naturalness, overall_naturalness

In [90]:
def reconstruct_sentence(original_sentence: str, alpha: float, verbose: bool=False) -> Tuple[int, int, int]:
    '''
    Returns position of the original sentence.
    '''
    original_sentence = preprocess(original_sentence)
    words = preprocess(original_sentence).split()
    
    permutations_list = list(permutations(words))
    naturalness_array = np.zeros((len(permutations_list), 3))  # in each row - naturalness from words, naturalness from tags, and overall one
    
    for i, sentence in enumerate(permutations_list):
        naturalness_array[i] = list(compute_sentence_naturalness(sentence=sentence, alpha=alpha))
    
    best_for_word_naturalness = np.argmax(naturalness_array[:, 0])
    best_for_tag_naturalness = np.argmax(naturalness_array[:, 1])
    best_overall_naturalness = np.argmax(naturalness_array[:, 2])
    
    if verbose:
        print(f'Best from word bigrams:\t{" ".join(permutations_list[best_for_word_naturalness])}')
        print(f'Best from tag bigrams:\t{" ".join(permutations_list[best_for_tag_naturalness])}')
        print(f'Best overall:\t{" ".join(permutations_list[best_overall_naturalness])}')
    
    idx_of_original = np.flatnonzero(np.array([' '.join(perm) for perm in permutations_list]) == original_sentence)[0]
    
    idx_from_word = np.flatnonzero(np.argsort(-naturalness_array[:, 0]) == idx_of_original)[0]
    idx_from_tag = np.flatnonzero(np.argsort(-naturalness_array[:, 1]) == idx_of_original)[0]
    idx_from_overall = np.flatnonzero(np.argsort(-naturalness_array[:, 2]) == idx_of_original)[0]
    
    return idx_from_word, idx_from_tag, idx_from_overall

In [115]:
def test_reconstructing():
    sentences = open('test_reconstruct_sents.txt', 'r').read().split('\n')
    acc = np.zeros((len(sentences), 3))
    for i, sentence in tqdm(enumerate(sentences), position=0, leave=True):
        words_rec, tags_rec, overall_rec = reconstruct_sentence(original_sentence=sentence, alpha=0.4)
        acc[i] = [words_rec, tags_rec, overall_rec]
    
    print(f'From word bigrams:\nscore: {np.mean(1 / (acc[:, 0] + 1))}\ngood ones: {np.count_nonzero(acc[:, 0] == 0)}\n')
    print(f'From tag bigrams:\nscore: {np.mean(1 / (acc[:, 1] + 1))}\ngood ones: {np.count_nonzero(acc[:, 1] == 0)}\n')
    print(f'From overall:\nscore: {np.mean(1 / (acc[:, 2] + 1))}\ngood ones: {np.count_nonzero(acc[:, 2] == 0)}\n')

In [116]:
test_reconstructing()

65it [00:02, 21.75it/s]

From word bigrams:
score: 0.23225804539576722
good ones: 12

From tag bigrams:
score: 0.25256706740700174
good ones: 12

From overall:
score: 0.2632054478187619
good ones: 12



### Zadanie 3. (4p) 
W zadaniu tym powinieneś losować zdania o słowach z identyczną charakterystyką gramatyczną jak zdanie wejściowe). Przykładowo dla zdania:

`Mały Piotruś spotkał w niewielkiej restauracyjce wczoraj poznaną koleżankę.` 
wynikiem mogłoby być
Gruby Stefan przeczytał we wczorajszej gazecie starannie przygotowaną analizę.
Zgodność gramatyczną sprawdzamy za pomocą tagów z pliku supertags. Przyjmijmy, że słowo s niewystępujące w tym pliku ma opis gramatyczny (’^’ + s)[-3:]. Powinieneś korzystać ze statystyk unigramowych.

In [8]:
def create_tag_to_words_mapping() -> DefaultDict[str, List[str]]:
    tag_to_words_mapping = defaultdict(list)
    
    for word, tag in tqdm(word_to_tag_mapping.items(), desc='Creating tag to words mapping', position=0, leave=True):
        tag_to_words_mapping[tag].append(word)
    return tag_to_words_mapping

In [9]:
word_counts = pickle.load(open('data.nogit/corpora_counts.pkl', 'rb'))

In [10]:
tag_to_words_mapping = create_tag_to_words_mapping()

Creating tag to words mapping: 100%|██████████| 1781994/1781994 [00:05<00:00, 340764.06it/s]


In [98]:
def create_similar_sentence(original_sentence: str) -> str:
    result = ''
    preprocessed = preprocess(original_sentence)
    original_words = preprocessed.split()
    
    for word in original_words:
        if word in word_to_tag_mapping:
            tag = word_to_tag_mapping[word]
        else:
            tag = word_to_tag_mapping['^'+s[-3:]]
        
        possible_words = tag_to_words_mapping[tag]
        probs = np.array([word_counts[possible_word] for possible_word in possible_words])
        probs /= probs.sum()
        next_word = np.random.choice(possible_words, p=probs)
        result += next_word + ' '
    return result

In [141]:
orignal_sentences = [
    'Mały Piotruś spotkał w niewielkiej restauracyjce wczoraj poznaną koleżankę.',
    'Pani postanowiła przejść się po niewielkim parku.',
    'Niemiecki lekarz prowadzi sportowy samochód.',
]

In [142]:
for original_sentence in orignal_sentences:
    print(f'Original sentence: {original_sentence}\n')
    for _ in range(3):
        print(create_similar_sentence(original_sentence))
    print('\n', '-'*42, '\n')

Original sentence: Mały Piotruś spotkał w niewielkiej restauracyjce wczoraj poznaną koleżankę.

sympatyczny poprzednik utworzył pod nierdzewnej drużynie prawdopodobnie prowadzoną eksploatację 
piękny nabywca polecił na niepodległościowej galaktyce stosownie wzbogaconą rolę 
obywatelski organizator zaliczył w niewielkiej padlinie znakomicie przyznaną motywację 

 ------------------------------------------ 

Original sentence: Pani postanowiła przejść się po niewielkim parku.

pani rozpoczęła przejść się na niebywałym parku 
pani zajęła zdobyć się na niebezpiecznym parku 
pani doprowadziła zdobyć się w niezależnym parku 

 ------------------------------------------ 

Original sentence: Niemiecki lekarz prowadzi sportowy samochód.

nowy poseł mówi wolny projekt 
wąski prokurator kocha prawdziwy awans 
narodowy pan przyznaje umiarkowany udział 

 ------------------------------------------ 



### Zadanie 4. (3p) 
Dodaj statystyki bigramowe do powyższego zadania. Postaraj się, by jak najrzadziej zdażały się sytuacje, w których musisz losować posługując się unigramami. Zaznaczaj znakiem "|" każdą taką nieciągłość.

In [111]:
def create_similar_sentence_with_bigram(original_sentence: str) -> str:
    result = ''
    preprocessed = preprocess(original_sentence)
    original_words = preprocessed.split()
    
    previous_word = None
    
    for word in original_words:
        if word in word_to_tag_mapping:
            tag = word_to_tag_mapping[word]
        else:
            tag = word_to_tag_mapping['^'+s[-3:]]
        
        if previous_word is None:  # first word
            possible_words = tag_to_words_mapping[tag]
        else:
            possible_words = [word for word in tag_to_words_mapping[tag] if word in word_bigrams[previous_word]]
            
            if len(possible_words) == 0:
                result += ' | '
                possible_words = tag_to_words_mapping[tag]
                
        
        probs = np.array([word_counts[possible_word] for possible_word in possible_words])
        probs /= probs.sum()
        next_word = np.random.choice(possible_words, p=probs)
        result += next_word + ' '
        
        previous_word = word
    return result

In [138]:
for original_sentence in orignal_sentences:
    print(f'Original sentence: {original_sentence}\n')
    for _ in range(3):
        print(create_similar_sentence_with_bigram(original_sentence))
    print('\n', '-'*42, '\n')

Original sentence: Mały Piotruś spotkał w niewielkiej restauracyjce wczoraj poznaną koleżankę.

górny pan  | przyjął w niewielkiej górze  | około  | prognozowaną historię 
barski pan  | zyskał w nieuczciwej zmianie  | obiektywnie  | oznaczoną historię 
główny jan  | przekazał na niedostatecznej stracie  | wielce  | zabrudzoną historię 

 ------------------------------------------ 

Original sentence: Pani postanowiła przejść się po niewielkim parku.

pani skończyła zdobyć się w niezwykłym parku 
pani podjęła przejść się na niezbędnym parku 
pani powołała pozbyć się w niespodziewanym parku 

 ------------------------------------------ 

Original sentence: Niemiecki lekarz prowadzi sportowy samochód.

demokratyczny rzecznik podchodzi szeroki projekt 
właściwy minister określa każdy tłumik 
spektakularny malarz podkreśla inny projekt 

 ------------------------------------------ 



### Zadanie 5. (5+1p) 
W zadaniu tym zajmiemy się kolokacjami słów, o których mamy informacje gramatyczną. Napisz program, który dla danego słowa znajduje k najbardziej z nim „spokrewnionych” słów. Rozważ następujące metody wyznaczania kolokacji:

a) PPMI (Positive Pointwise Mutual Information)

b) jakiś inny, dowolnie wybrany, z używanych na kolokacje wzorów (więcej na wykładzie),

c) kolokacje gramatyczno-słowowe (tzn. żeby dwa słowa były uznane za kolokacje, warunek kolo- kacyjności (dowolnie wybrany) powinny spełniać zarówno tagi słów, jak i same słowa.

d) jakaś dowolna inna metoda, lub Twoja modyfikacja powyższych (za to dodatkowy punkt).

Możesz się ograniczyć do słów, które są stosunkowo częste (więcej niż n wystąpień w korpusie) i występują co najmniej raz w jakimś trigramie (lub k razy w jakimś bigramie). Wybierz niewielki zbiór słów (powiedzmy koło 10). Przygotuj raport, w którym dla każdego z tych słów jest 10 najbardziej spokrewnionych słow (czyli takich, o największym współczynniku kolokacji), dla różnych metod wyznaczania kolokacji.

In [67]:
def PPMI(
    word1: str, 
    word2: str, 
    bigrams_count: DefaultDict[Tuple[str, str], float],
    word_counts: DefaultDict[str, float],
) -> float:
    prob_word1_word2 = bigrams_count[word1, word2]
    prob_word1 = word_counts[word1]
    prob_word2 = word_counts[word2]
    
    return np.log(1e-10 + (prob_word1_word2 / (1e-10 + (prob_word1 * prob_word2))))

In [103]:
def PSM(word1: str, word2: str) -> float:
    N = 26544910
    
    prob_bigram_w1w2 = word_bigrams_count[word1, word2]
    bigram_w1w2_count = prob_bigram_w1w2
    
    w1_count = word_counts[word1]
    w2_count = word_counts[word2]
    
    
    return bigram_w1w2_count * (np.log(1e-10 + bigram_w1w2_count) - np.log(1e-10 + (w1_count * w2_count / N)) - 1)

In [44]:
tag_counts = defaultdict(float)
for word, count in tqdm(word_counts.items(), position=0, leave=True):
    if word in word_to_tag_mapping:
        tag = word_to_tag_mapping[word]
        tag_counts[tag] += count

100%|██████████| 2980295/2980295 [00:08<00:00, 359714.14it/s]


In [68]:
def PPMI_with_tags(
    word1: str, 
    word2: str,
) -> float:
    ppmi_w = PPMI(word1, word2, word_bigrams_count, word_counts)
    
    if word1 not in word_to_tag_mapping or word2 not in word_to_tag_mapping:
        return ppmi_w
    
    tag1, tag2 = word_to_tag_mapping[word1], word_to_tag_mapping[word2]
    ppmi_t = PPMI(tag1, tag2, tag_bigrams_count, tag_counts)
    
    return ppmi_w + ppmi_t

In [101]:
def PPMI_alpha(word1: str, word2: str, alpha: float):
    pw1 = word_bigrams_count[word1] ** 0.75
    pw1w2 = word_bigrams_count[word1, word2] ** 0.75
    pw2 = word_bigrams_count[word2] ** 0.75
    N = 26544910
    return np.log(1e-10 + (N*pw1w2)/(pw1*pw2 + 1e-10))

In [79]:
min_prob = 50 / 26544910
selected_words = [w for w, p in word_counts.items() if p > min_prob]

In [80]:
len(selected_words)

39091

In [91]:
def print_k_nearest(word: str, k: int) -> None:
    print(f'FOR WORD: {word}')
    
    ppmi_values = list(map(lambda w: PPMI(word, w, word_bigrams_count, word_counts), selected_words))
    ppmi_best_words = np.argsort(ppmi_values)[::-1][:k]
    print('\n\t\t\tPPMI')
    for idx in ppmi_best_words:
        print(f'{selected_words[idx]}\t-\t{ppmi_values[idx]}')
    
    psm_values = list(map(lambda w: PSM(word, w), selected_words))
    psm_best_words = np.argsort(psm_values)[::-1][:k]
    print('\n\t\t\tPSM')
    for idx in psm_best_words:
        print(f'{selected_words[idx]}\t-\t{psm_values[idx]}')
    
    
    ppmi_with_tags_values = list(map(lambda w: PPMI_with_tags(word, w), selected_words))
    ppmi_with_tags_best_words = np.argsort(ppmi_with_tags_values)[::-1][:k]
    print('\n\t\t\tPPMI with tags')
    for idx in ppmi_with_tags_best_words:
        print(f'{selected_words[idx]}\t-\t{ppmi_with_tags_values[idx]}')
    
    ppmi_alpha_values = list(map(lambda w: PPMI_alpha(word, w, 0.75), selected_words))
    ppmi_alpha_best_words = np.argsort(ppmi_alpha_values)[::-1][:k]
    print('\n\t\t\tPPMI alpha - 0.75')
    for idx in ppmi_alpha_best_words:
        print(f'{selected_words[idx]}\t-\t{ppmi_alpha_values[idx]}')
    

In [105]:
words = ['biegałem', 'ten', 'park', 'jaki', 'nie']

for word in words:
    print_k_nearest(word, k=5)
    print('\n\n', '-'*42, '\n\n')

FOR WORD: biegałem

			PPMI
po	-	19.004970047163397
bez	-	18.86430467725975
za	-	17.77402384167
z	-	17.34831753850049
na	-	16.792672784516945

			PSM
po	-	2.980193800334684
z	-	2.590359805678896
w	-	2.5903595603228076
na	-	1.4387525823783303
i	-	1.4387525554686387

			PPMI with tags
po	-	28.686147429179712
za	-	27.843614538855736
z	-	27.36893177840038
bez	-	27.126413994653042
na	-	26.47385016653326

			PPMI alpha - 0.75
po	-	38.68804262480227
w	-	38.58789408033388
z	-	38.58789408033388
i	-	38.16818223938232
na	-	38.16818223938232


 ------------------------------------------ 


FOR WORD: ten

			PPMI
zdiagnozował	-	13.043566210184684
cel	-	12.446343397814632
opisywany	-	11.91871967985291
proceder	-	11.889111524414153
powtórzył	-	11.766689566484004

			PSM
sposób	-	1.9082635670067443
projekt	-	0.8961745270380157
temat	-	0.8412243997498091
cel	-	0.6258829716390054
jest	-	0.46349774166070645

			PPMI with tags
sposób	-	21.786011888161617
cel	-	21.242631795972663
sam	-	21.127682642795424
z

### Zadanie 6. (7p) 
W zadaniu tym będziemy tworzyć pierwszą wersję programu tworzącego poezję (przypominającą Pana Tadeusza, oznaczanego dalej PT)). Przypomnijmy najważniejsze fakty odnoszące się do tego utworu (i ogólnie Poezji):

F1. Wiersz składa się z wersetów, z których każdy ma ustaloną liczbę sylab (w PT to 13)

F2. Ostatnie słowo w wersecie n rymuje się z ostatnim słowem w wersecie n + 1 (dla n parzystego,
numeracja od 0).

F3. Rym to zgodność ostatniej sylaby i części od samogłoski sylaby przedostatniej (zdrowie-dowie). W zasadzie rymy to zjawisko fonetyczne, ale dla języka polskiego (w pierwszej wersji) można sobie to trochę uprościć i powiedzieć, że dotyczą one liter.

F4. Akcenty słów i podziały słów muszą się jakoś sensownie układać. Nam wystarczy przyjąć, że godzimy się jedynie na takie podziały wersu na słowa k-sylabowe3, których użył Adam Mickiewicz, na przykład:

Litwo, Ojczyzno moja, Ty jesteś jak zdrowie, ile cię trzeba cenić, ten tylko się dowie ma schemat: [2,3,2,1,2,1,2] -- [2,1,2,2,1,2,1,2]

F5. Podział na sylaby nie jest trywialny (dlaczego?). Ale na nasze szczęście policzenie sylab jest łatwe. Słowo ma tyle sylab, ile ma samogłosek (przy czym połączenia ie, iu, ię, itd traktujemy jako jedną samogłoskę). Część rymowana wyrazu to część wyrazu od przedostatniej samogłoski do końca.

Powinieneś stworzyć program, generujący dwuwersowe fragmenty wierszy w stylu PT, czyli powinieneś przypilnować:

a) żeby wersy były poprawne rytmicznie i się rymowały,

b) żeby dwuwers miał sens gramatyczny (czyli by tagi słów pasowały do jakiegoś zdania lub frag- mentu zdania)

c) żeby były jakoś wykorzystane statystyki N-gramowe (plan minimum to statystyki 1-gramowe, dodatkowe +1 za wykorzystanie bigramów).

### Zadanie 7. (5p) 
Zmodyfikuj algorytm z poprzedniego zadania w ten sposób, by starał się on maksymalizować wzajemną „kolokacyjność” słów z dwuwersu (tak, by jak najwięcej słów było ze sobą powiązanych, na przykład przez wysokie PPMI). Akceptowalna jest dowolna procedura (local search, jakieś błądzenie losowe, metody ewolucyjne, ...), która daje wartość liczby par słów będących kolokacjami istotnie większą niż losowanie z poprzedniego zadania.
